# 데이터 불러오기

## 필수 라이브러리

In [3]:
import pandas as pd
import os

/Users/sojeonglee/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## 데이터 읽어오기

In [4]:
ROOT_DIR = "data"
RANDOM_STATE = 110

# Load data
df_news2 = pd.read_csv(os.path.join(ROOT_DIR, "news2.csv"))
df_news2

Unnamed: 0                                              title  \
0           8476                       You Can Smell Hillary’s Fear   
1          10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2           3608        Kerry to go to Paris in gesture of sympathy   
3          10142  Bernie supporters on Twitter erupt in anger ag...   
4            875   The Battle of New York: Why This Primary Matters   
...          ...                                                ...   
6330        4490  State Department says it can't find emails fro...   
6331        8062  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...   
6332        8622  Anti-Trump Protesters Are Tools of the Oligarc...   
6333        4021  In Ethiopia, Obama seeks progress on peace, se...   
6334        4330  Jeb Bush Is Suddenly Attacking Trump. Here's W...   

                                                   text label  
0     Daniel Greenfield, a Shillman Journalism Fello...  FAKE  
1     Google Pinterest Digg Linkedin Reddit Stumbleu...  FAKE  
2     U.S. Secretary of State John F. Kerry said Mon...  REAL  
3     — Kaydee King (@KaydeeKing) November 9, 2016 T...  FAKE  
4     It's primary day in New York and front-runners...  REAL  
...                                                 ...   ...  
6330  The State Department told the Republican Natio...  REAL  
6331  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...  FAKE  
6332   Anti-Trump Protesters Are Tools of the Oligar...  FAKE  
6333  ADDIS ABABA, Ethiopia —President Obama convene...  REAL  
6334  Jeb Bush Is Suddenly Attacking Trump. Here's W...  REAL  

[6335 rows x 4 columns]

# Data preprocessing

## remove Nan data

In [5]:
nan_cols = []
for col in df_news2.columns:
    if (df_news2[col].isnull().all()):
        nan_cols.append(col)
df_train = df_news2.drop(nan_cols, axis=1)
df_test = df_news2.drop(nan_cols, axis=1)
print(nan_cols)

[]


## remove constant data

In [6]:
constant_col = []
for col in df_news2.columns :
    if df_news2[col].nunique() == 1 :
        constant_col.append(col)
X = df_news2.drop(constant_col, axis=1)
print(constant_col)

[]


## replace Nan value

In [7]:
for col in df_news2.columns :
    if df_news2[col].isnull().sum() !=0 :
        print(col, df_news2[col].isnull().sum())

In [8]:
df_news2 = df_news2.fillna(0)

## fake = 0, real = 1

In [9]:
df_news2.loc[df_news2['label'] == 'FAKE', 'label'] = 0.0
df_news2.loc[df_news2['label'] == 'REAL', 'label'] = 1.0

In [10]:
df_news2

Unnamed: 0                                              title  \
0           8476                       You Can Smell Hillary’s Fear   
1          10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2           3608        Kerry to go to Paris in gesture of sympathy   
3          10142  Bernie supporters on Twitter erupt in anger ag...   
4            875   The Battle of New York: Why This Primary Matters   
...          ...                                                ...   
6330        4490  State Department says it can't find emails fro...   
6331        8062  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...   
6332        8622  Anti-Trump Protesters Are Tools of the Oligarc...   
6333        4021  In Ethiopia, Obama seeks progress on peace, se...   
6334        4330  Jeb Bush Is Suddenly Attacking Trump. Here's W...   

                                                   text label  
0     Daniel Greenfield, a Shillman Journalism Fello...   0.0  
1     Google Pinterest Digg Linkedin Reddit Stumbleu...   0.0  
2     U.S. Secretary of State John F. Kerry said Mon...   1.0  
3     — Kaydee King (@KaydeeKing) November 9, 2016 T...   0.0  
4     It's primary day in New York and front-runners...   1.0  
...                                                 ...   ...  
6330  The State Department told the Republican Natio...   1.0  
6331  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...   0.0  
6332   Anti-Trump Protesters Are Tools of the Oligar...   0.0  
6333  ADDIS ABABA, Ethiopia —President Obama convene...   1.0  
6334  Jeb Bush Is Suddenly Attacking Trump. Here's W...   1.0  

[6335 rows x 4 columns]

## tokenize

In [12]:
# vocab loading
import sentencepiece as spm
vocab_file = "news2.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

True

### tokenize ; downloaded as json form

In [13]:
""" train data 준비 """
def prepare_train(vocab, infile, outfile):
    df = pd.read_csv(infile, sep="\t", engine="python")
    with open(outfile, "w") as f:
        for index, row in df.iterrows():
            document = row["document"]
            if type(document) != str:
                continue
            instance = { "id": row["id"], "doc": vocab.encode_as_pieces(document), "label": row["label"] }
            f.write(json.dumps(instance))
            f.write("\n")

In [ ]:
#prepare_train(vocab, "<path of data>/ratings_train.txt", "<path of data>/ratings_train.json")
#prepare_train(vocab, "<path of data>/ratings_test.txt", "<path of data>/ratings_test.json")